In [1]:
%load_ext autoreload
%autoreload 2
import xarray as xr
import matplotlib.pyplot as plt
import pandas as pd
import processing_netcdf as pcdf
import geopandas as gpd
from geopandas import GeoDataFrame
import shapely.geometry 
import numpy as np
from shapely import geometry as gmty
from geofeather import to_geofeather, from_geofeather
import glob
import os
import pyarrow
from xclim import ensembles as ens
from xclim import subset
import matplotlib.pyplot as plt

# Environment 

In [2]:
#Select Variable 
var = "growing_season_length"
#NorESM1-M_rcp45_solidprcptot_monthly

# Run all from here ############

In [3]:
folder = "/scen3/scenario/netcdf/ouranos/portraits-clim-2.0/"

In [4]:
variable85="rcp85_"+var+"_annual"
variable45="rcp45_"+var+"_annual"
#variable="rcp[48]5_tg_mean_annual"

files85 = glob.glob(folder+"*"+variable85+".nc")
files45 = glob.glob(folder+"*"+variable45+".nc")
#ex: ACCESS1-3_rcp45_tn_mean_annual.nc
files85
files45

['/scen3/scenario/netcdf/ouranos/portraits-clim-2.0/ACCESS1-3_rcp45_growing_season_length_annual.nc',
 '/scen3/scenario/netcdf/ouranos/portraits-clim-2.0/BNU-ESM_rcp45_growing_season_length_annual.nc',
 '/scen3/scenario/netcdf/ouranos/portraits-clim-2.0/CanESM2_rcp45_growing_season_length_annual.nc',
 '/scen3/scenario/netcdf/ouranos/portraits-clim-2.0/CMCC-CMS_rcp45_growing_season_length_annual.nc',
 '/scen3/scenario/netcdf/ouranos/portraits-clim-2.0/GFDL-ESM2M_rcp45_growing_season_length_annual.nc',
 '/scen3/scenario/netcdf/ouranos/portraits-clim-2.0/INM-CM4_rcp45_growing_season_length_annual.nc',
 '/scen3/scenario/netcdf/ouranos/portraits-clim-2.0/IPSL-CM5A-LR_rcp45_growing_season_length_annual.nc',
 '/scen3/scenario/netcdf/ouranos/portraits-clim-2.0/IPSL-CM5B-LR_rcp45_growing_season_length_annual.nc',
 '/scen3/scenario/netcdf/ouranos/portraits-clim-2.0/MPI-ESM-LR_rcp45_growing_season_length_annual.nc',
 '/scen3/scenario/netcdf/ouranos/portraits-clim-2.0/NorESM1-M_rcp45_growing_seaso

# FOR RCP 45

## Subsetting arrays by periods of time and percentiles with Xclim

In [5]:
dsEns45= ens.create_ensemble(files45)
dsEns45

<xarray.Dataset>
Dimensions:                (lat: 320, lon: 416, realization: 10, time: 151)
Coordinates:
  * lat                    (lat) float32 66.62331 66.53998 ... 40.12437 40.04104
  * time                   (time) datetime64[ns] 1950-01-01 ... 2100-01-01
  * lon                    (lon) float32 -89.04521 -88.96188 ... -54.46326
  * realization            (realization) int64 0 1 2 3 4 5 6 7 8 9
Data variables:
    growing_season_length  (realization, time, lat, lon) float64 dask.array<chunksize=(1, 10, 320, 416), meta=np.ndarray>
Attributes:
    Conventions:     CF-1.5
    title:           ACCESS1-3 model output prepared for CMIP5 historical
    history:         CMIP5 compliant file produced from raw ACCESS model outp...
    institution:     CSIRO (Commonwealth Scientific and Industrial Research O...
    source:          ACCESS1-3 2011. Atmosphere: AGCM v1.0 (N96 grid-point, 1...
    redistribution:  Redistribution prohibited. For internal use only.

In [6]:
# Mapping data - 30 year means
tmp145 = dsEns45.sel(time=(dsEns45.time.dt.year>=1981))
window = 30
time145 = tmp145.time[0::window]

ds30yavg45 = tmp145.coarsen(time=window).mean()
ds30yavg45['time'] = time145
perc30yavg45 = ens.ensemble_percentiles(ds30yavg45)
perc30yavg45 # entire grid

# Mapping data

<xarray.Dataset>
Dimensions:                    (lat: 320, lon: 416, realization: 10, time: 4)
Coordinates:
  * lat                        (lat) float32 66.62331 66.53998 ... 40.04104
  * time                       (time) datetime64[ns] 1981-01-01 ... 2071-01-01
  * lon                        (lon) float32 -89.04521 -88.96188 ... -54.46326
  * realization                (realization) int64 0 1 2 3 4 5 6 7 8 9
Data variables:
    growing_season_length_p10  (time, lat, lon) float64 dask.array<chunksize=(1, 320, 416), meta=np.ndarray>
    growing_season_length_p50  (time, lat, lon) float64 dask.array<chunksize=(1, 320, 416), meta=np.ndarray>
    growing_season_length_p90  (time, lat, lon) float64 dask.array<chunksize=(1, 320, 416), meta=np.ndarray>

## Transform into DataFrame

In [7]:
df450 = perc30yavg45.drop('realization').to_dataframe().dropna()

df45 = df450.reset_index()

df45.to_feather("/home/mlopez/EXEC/Processed data to clip with regions/"+var+"df45_anual.feather")

  x = np.divide(x1, x2, out)

  x = np.divide(x1, x2, out)

  x = np.divide(x1, x2, out)

  x = np.divide(x1, x2, out)

  x = np.divide(x1, x2, out)

  x = np.divide(x1, x2, out)



In [14]:
df45 = pd.read_feather("/home/mlopez/EXEC/Processed data to clip with regions/"+var+"df45_anual.feather")

## Transform in Celsius and round

In [15]:
df45C = df45.copy()
df45C[var+"_p10"] = round(df45C[var+"_p10"], 2)
df45C[var+"_p50"] = round(df45C[var+"_p50"], 2)
df45C[var+"_p90"] = round(df45C[var+"_p90"], 2)

df45C

,lat,lon,time,growing_season_length_p10,growing_season_length_p50,growing_season_length_p90
0,66.623306,-89.045212,1981-01-01,43.80,48.97,55.32
1,66.623306,-89.045212,2011-01-01,46.40,61.14,69.43
2,66.623306,-89.045212,2041-01-01,55.50,70.38,80.53
3,66.623306,-89.045212,2071-01-01,54.25,75.30,86.83
4,66.623306,-88.961884,1981-01-01,45.14,49.74,55.68
...,...,...,...,...,...,...
304819,40.041039,-74.129143,2071-01-01,288.25,308.10,311.26
304820,40.041039,-74.045807,1981-01-01,270.13,273.23,279.61
304821,40.041039,-74.045807,2011-01-01,283.50,292.83,298.55
304822,40.041039,-74.045807,2041-01-01,287.09,298.80,307.29


## Eliminate 2011, Pivot Table and Create new columns 

In [16]:
Region1i45 = df45C
print (Region1i45)
Region1i245 = Region1i45[Region1i45.time.dt.year!= 2011]
print ("2011 eliminated")
df45f = Region1i245.pivot_table(index=["lat","lon"], columns="time")
print ("DF pivoted")
df45f.columns = [year + "rcp45_p"+p
                     for p in ["10", "50", "90"]
                     for year in ["hist_","t2050_", "t2080_"]]
print("Columns renamed")
df45f

              lat        lon       time  growing_season_length_p10  \
0       66.623306 -89.045212 1981-01-01                      43.80   
1       66.623306 -89.045212 2011-01-01                      46.40   
2       66.623306 -89.045212 2041-01-01                      55.50   
3       66.623306 -89.045212 2071-01-01                      54.25   
4       66.623306 -88.961884 1981-01-01                      45.14   
...           ...        ...        ...                        ...   
304819  40.041039 -74.129143 2071-01-01                     288.25   
304820  40.041039 -74.045807 1981-01-01                     270.13   
304821  40.041039 -74.045807 2011-01-01                     283.50   
304822  40.041039 -74.045807 2041-01-01                     287.09   
304823  40.041039 -74.045807 2071-01-01                     288.83   

        growing_season_length_p50  growing_season_length_p90  
0                           48.97                      55.32  
1                           61.14

hist_rcp45_p10  t2050_rcp45_p10  t2080_rcp45_p10  \
lat       lon                                                            
40.041039 -89.045212          251.13           266.38           269.31   
          -88.961884          250.62           265.77           269.27   
          -88.878548          250.77           265.70           269.24   
          -88.795219          250.07           266.11           268.17   
          -88.711891          250.96           265.31           268.28   
...                              ...              ...              ...   
66.623306 -61.629639           30.90            32.51            24.05   
          -61.546310           27.33            26.65            19.73   
          -61.462978           30.71            30.98            23.85   
          -61.379650           31.01            31.34            24.34   
          -61.296322           37.11            47.15            38.16   

                      hist_rcp45_p50  t2050_rcp45_p50  t2080_rcp45_p50  \
lat       lon                                                            
40.041039 -89.045212          256.57           271.37           279.70   
          -88.961884          255.27           271.17           279.63   
          -88.878548          255.27           270.90           279.63   
          -88.795219          254.57           270.20           279.33   
          -88.711891          254.53           271.23           278.77   
...                              ...              ...              ...   
66.623306 -61.629639           36.39            48.56            53.31   
          -61.546310           34.06            42.81            47.12   
          -61.462978           34.06            47.62            51.06   
          -61.379650           34.61            47.96            51.25   
          -61.296322           47.12            55.59            60.29   

                      hist_rcp45_p90  t2050_rcp45_p90  t2080_rcp45_p90  
lat       lon                                                           
40.041039 -89.045212          258.48           287.71           290.23  
          -88.961884          258.26           287.65           290.40  
          -88.878548          258.68           287.71           290.39  
          -88.795219          258.39           287.93           290.14  
          -88.711891          257.14           286.49           290.78  
...                              ...              ...              ...  
66.623306 -61.629639           43.07            67.11            64.54  
          -61.546310           35.98            65.65            59.60  
          -61.462978           40.44            67.05            62.40  
          -61.379650           41.76            68.02            62.53  
          -61.296322           52.51            69.59            69.60  

[76206 rows x 9 columns]

# FOR RCP 85

## Subsetting arrays by periods of time and percentiles with Xclim

In [6]:
dsEns85= ens.create_ensemble(files85)
dsEns85

<xarray.Dataset>
Dimensions:                (lat: 320, lon: 416, realization: 11, time: 151)
Coordinates:
  * time                   (time) datetime64[ns] 1950-01-01 ... 2100-01-01
  * lat                    (lat) float32 66.62331 66.53998 ... 40.12437 40.04104
  * lon                    (lon) float32 -89.04521 -88.96188 ... -54.46326
  * realization            (realization) int64 0 1 2 3 4 5 6 7 8 9 10
Data variables:
    growing_season_length  (realization, time, lat, lon) float64 dask.array<chunksize=(1, 10, 320, 416), meta=np.ndarray>
Attributes:
    Conventions:     CF-1.5
    title:           ACCESS1-3 model output prepared for CMIP5 historical
    history:         CMIP5 compliant file produced from raw ACCESS model outp...
    institution:     CSIRO (Commonwealth Scientific and Industrial Research O...
    source:          ACCESS1-3 2011. Atmosphere: AGCM v1.0 (N96 grid-point, 1...
    redistribution:  Redistribution prohibited. For internal use only.

In [7]:
# Mapping data - 30 year means
tmp185 = dsEns85.sel(time=(dsEns85.time.dt.year>=1981))
window = 30
time185 = tmp185.time[0::window]

ds30yavg85 = tmp185.coarsen(time=window).mean()
ds30yavg85['time'] = time185
perc30yavg85 = ens.ensemble_percentiles(ds30yavg85)
perc30yavg85 # entire grid

<xarray.Dataset>
Dimensions:                    (lat: 320, lon: 416, realization: 11, time: 4)
Coordinates:
  * time                       (time) datetime64[ns] 1981-01-01 ... 2071-01-01
  * lat                        (lat) float32 66.62331 66.53998 ... 40.04104
  * lon                        (lon) float32 -89.04521 -88.96188 ... -54.46326
  * realization                (realization) int64 0 1 2 3 4 5 6 7 8 9 10
Data variables:
    growing_season_length_p10  (time, lat, lon) float64 dask.array<chunksize=(1, 320, 416), meta=np.ndarray>
    growing_season_length_p50  (time, lat, lon) float64 dask.array<chunksize=(1, 320, 416), meta=np.ndarray>
    growing_season_length_p90  (time, lat, lon) float64 dask.array<chunksize=(1, 320, 416), meta=np.ndarray>

## Transform into DataFrame

In [8]:
df85 = perc30yavg85.drop('realization').to_dataframe().dropna()
df85

  x = np.divide(x1, x2, out)

  x = np.divide(x1, x2, out)

  x = np.divide(x1, x2, out)

  x = np.divide(x1, x2, out)

  x = np.divide(x1, x2, out)

  x = np.divide(x1, x2, out)



growing_season_length_p10  \
lat       lon        time                                    
66.623306 -89.045212 1981-01-01                  41.777778   
                     2011-01-01                  49.035714   
                     2041-01-01                  62.655172   
                     2071-01-01                  76.500000   
          -88.961884 1981-01-01                  42.965517   
...                                                    ...   
40.041039 -74.129143 2071-01-01                 309.533333   
          -74.045807 1981-01-01                 266.066667   
                     2011-01-01                 282.200000   
                     2041-01-01                 298.566667   
                     2071-01-01                 309.533333   

                                 growing_season_length_p50  \
lat       lon        time                                    
66.623306 -89.045212 1981-01-01                  48.148148   
                     2011-01-01                  57.655172   
                     2041-01-01                  71.700000   
                     2071-01-01                 101.066667   
          -88.961884 1981-01-01                  49.433333   
...                                                    ...   
40.041039 -74.129143 2071-01-01                 329.366667   
          -74.045807 1981-01-01                 273.133333   
                     2011-01-01                 289.933333   
                     2041-01-01                 311.133333   
                     2071-01-01                 331.300000   

                                 growing_season_length_p90  
lat       lon        time                                   
66.623306 -89.045212 1981-01-01                  53.260870  
                     2011-01-01                  68.700000  
                     2041-01-01                  87.200000  
                     2071-01-01                 115.133333  
          -88.961884 1981-01-01                  53.592593  
...                                                    ...  
40.041039 -74.129143 2071-01-01                 344.666667  
          -74.045807 1981-01-01                 278.100000  
                     2011-01-01                 295.866667  
                     2041-01-01                 319.266667  
                     2071-01-01                 344.666667  

[304824 rows x 3 columns]

In [9]:
df85 = df85.reset_index()

df85.to_feather("/home/mlopez/EXEC/Processed data to clip with regions/"+var+"df85_anual.feather")

In [10]:
#df85 = pd.read_feather("/home/mlopez/EXEC/Processed data to clip with regions/"+var+"df85_anual.feather")

## Transform in Celsius and round

In [11]:
df85C = df85.copy()
df85C[var+"_p10"] = round(df85C[var+"_p10"], 2)
df85C[var+"_p50"] = round(df85C[var+"_p50"], 2)
df85C[var+"_p90"] = round(df85C[var+"_p90"], 2)
df85C

,lat,lon,time,growing_season_length_p10,growing_season_length_p50,growing_season_length_p90
0,66.623306,-89.045212,1981-01-01,41.78,48.15,53.26
1,66.623306,-89.045212,2011-01-01,49.04,57.66,68.70
2,66.623306,-89.045212,2041-01-01,62.66,71.70,87.20
3,66.623306,-89.045212,2071-01-01,76.50,101.07,115.13
4,66.623306,-88.961884,1981-01-01,42.97,49.43,53.59
...,...,...,...,...,...,...
304819,40.041039,-74.129143,2071-01-01,309.53,329.37,344.67
304820,40.041039,-74.045807,1981-01-01,266.07,273.13,278.10
304821,40.041039,-74.045807,2011-01-01,282.20,289.93,295.87
304822,40.041039,-74.045807,2041-01-01,298.57,311.13,319.27


## Eliminate 2011, Pivot Table and Create new columns 

In [12]:
Region1i85 = df85C
print (Region1i85)
Region1i285 = Region1i85[Region1i85.time.dt.year!= 2011]
print ("2011 eliminated")
df85f = Region1i285.pivot_table(index=["lat","lon"], columns="time")
print ("DF pivoted")
df85f.columns = [year + "rcp85_p"+p
                     for p in ["10", "50", "90"]
                     for year in ["hist_","t2050_", "t2080_"]]
print("Columns renamed")
df85f

              lat        lon       time  growing_season_length_p10  \
0       66.623306 -89.045212 1981-01-01                      41.78   
1       66.623306 -89.045212 2011-01-01                      49.04   
2       66.623306 -89.045212 2041-01-01                      62.66   
3       66.623306 -89.045212 2071-01-01                      76.50   
4       66.623306 -88.961884 1981-01-01                      42.97   
...           ...        ...        ...                        ...   
304819  40.041039 -74.129143 2071-01-01                     309.53   
304820  40.041039 -74.045807 1981-01-01                     266.07   
304821  40.041039 -74.045807 2011-01-01                     282.20   
304822  40.041039 -74.045807 2041-01-01                     298.57   
304823  40.041039 -74.045807 2071-01-01                     309.53   

        growing_season_length_p50  growing_season_length_p90  
0                           48.15                      53.26  
1                           57.66

hist_rcp85_p10  t2050_rcp85_p10  t2080_rcp85_p10  \
lat       lon                                                            
40.041039 -89.045212          248.40           276.70           290.30   
          -88.961884          248.40           276.70           290.30   
          -88.878548          248.77           277.30           290.27   
          -88.795219          246.93           277.47           289.13   
          -88.711891          245.67           276.70           289.13   
...                              ...              ...              ...   
66.623306 -61.629639           32.70            26.72            34.82   
          -61.546310           31.42            19.67            29.57   
          -61.462978           33.42            21.84            33.30   
          -61.379650           34.33            23.17            34.82   
          -61.296322           40.55            36.26            45.48   

                      hist_rcp85_p50  t2050_rcp85_p50  t2080_rcp85_p50  \
lat       lon                                                            
40.041039 -89.045212          253.63           282.30           303.87   
          -88.961884          253.60           282.33           304.07   
          -88.878548          253.17           282.37           304.07   
          -88.795219          253.13           281.13           304.60   
          -88.711891          253.53           282.43           304.87   
...                              ...              ...              ...   
66.623306 -61.629639           40.89            48.25            62.40   
          -61.546310           38.13            47.32            61.43   
          -61.462978           39.65            49.47            61.80   
          -61.379650           41.90            50.26            62.55   
          -61.296322           48.72            58.48            70.78   

                      hist_rcp85_p90  t2050_rcp85_p90  t2080_rcp85_p90  
lat       lon                                                           
40.041039 -89.045212          256.87           297.30           317.67  
          -88.961884          257.13           297.20           317.53  
          -88.878548          257.03           297.20           317.47  
          -88.795219          256.37           296.20           316.10  
          -88.711891          256.43           295.53           316.27  
...                              ...              ...              ...  
66.623306 -61.629639           49.55            69.71            84.24  
          -61.546310           45.90            66.39            82.93  
          -61.462978           47.04            69.29            83.79  
          -61.379650           49.33            69.75            83.00  
          -61.296322           55.19            76.17            90.43  

[76206 rows x 9 columns]

# Merge df45 and 85

In [17]:
dftg = pd.merge(df45f, df85f, on=["lat","lon"])
dftg.head()

hist_rcp45_p10  t2050_rcp45_p10  t2080_rcp45_p10  \
lat       lon                                                            
40.041039 -89.045212          251.13           266.38           269.31   
          -88.961884          250.62           265.77           269.27   
          -88.878548          250.77           265.70           269.24   
          -88.795219          250.07           266.11           268.17   
          -88.711891          250.96           265.31           268.28   

                      hist_rcp45_p50  t2050_rcp45_p50  t2080_rcp45_p50  \
lat       lon                                                            
40.041039 -89.045212          256.57           271.37           279.70   
          -88.961884          255.27           271.17           279.63   
          -88.878548          255.27           270.90           279.63   
          -88.795219          254.57           270.20           279.33   
          -88.711891          254.53           271.23           278.77   

                      hist_rcp45_p90  t2050_rcp45_p90  t2080_rcp45_p90  \
lat       lon                                                            
40.041039 -89.045212          258.48           287.71           290.23   
          -88.961884          258.26           287.65           290.40   
          -88.878548          258.68           287.71           290.39   
          -88.795219          258.39           287.93           290.14   
          -88.711891          257.14           286.49           290.78   

                      hist_rcp85_p10  t2050_rcp85_p10  t2080_rcp85_p10  \
lat       lon                                                            
40.041039 -89.045212          248.40           276.70           290.30   
          -88.961884          248.40           276.70           290.30   
          -88.878548          248.77           277.30           290.27   
          -88.795219          246.93           277.47           289.13   
          -88.711891          245.67           276.70           289.13   

                      hist_rcp85_p50  t2050_rcp85_p50  t2080_rcp85_p50  \
lat       lon                                                            
40.041039 -89.045212          253.63           282.30           303.87   
          -88.961884          253.60           282.33           304.07   
          -88.878548          253.17           282.37           304.07   
          -88.795219          253.13           281.13           304.60   
          -88.711891          253.53           282.43           304.87   

                      hist_rcp85_p90  t2050_rcp85_p90  t2080_rcp85_p90  
lat       lon                                                           
40.041039 -89.045212          256.87           297.30           317.67  
          -88.961884          257.13           297.20           317.53  
          -88.878548          257.03           297.20           317.47  
          -88.795219          256.37           296.20           316.10  
          -88.711891          256.43           295.53           316.27

# Extract hist

In [18]:
dftg1 = dftg.drop(columns=["hist_rcp45_p10","hist_rcp45_p50", "hist_rcp45_p90", "hist_rcp85_p10", "hist_rcp85_p50", "hist_rcp85_p90"])
dftgh = dftg[["hist_rcp45_p50", "hist_rcp85_p50"]]

In [19]:
dftg1

t2050_rcp45_p10  t2080_rcp45_p10  t2050_rcp45_p50  \
lat       lon                                                             
40.041039 -89.045212           266.38           269.31           271.37   
          -88.961884           265.77           269.27           271.17   
          -88.878548           265.70           269.24           270.90   
          -88.795219           266.11           268.17           270.20   
          -88.711891           265.31           268.28           271.23   
...                               ...              ...              ...   
66.623306 -61.629639            32.51            24.05            48.56   
          -61.546310            26.65            19.73            42.81   
          -61.462978            30.98            23.85            47.62   
          -61.379650            31.34            24.34            47.96   
          -61.296322            47.15            38.16            55.59   

                      t2080_rcp45_p50  t2050_rcp45_p90  t2080_rcp45_p90  \
lat       lon                                                             
40.041039 -89.045212           279.70           287.71           290.23   
          -88.961884           279.63           287.65           290.40   
          -88.878548           279.63           287.71           290.39   
          -88.795219           279.33           287.93           290.14   
          -88.711891           278.77           286.49           290.78   
...                               ...              ...              ...   
66.623306 -61.629639            53.31            67.11            64.54   
          -61.546310            47.12            65.65            59.60   
          -61.462978            51.06            67.05            62.40   
          -61.379650            51.25            68.02            62.53   
          -61.296322            60.29            69.59            69.60   

                      t2050_rcp85_p10  t2080_rcp85_p10  t2050_rcp85_p50  \
lat       lon                                                             
40.041039 -89.045212           276.70           290.30           282.30   
          -88.961884           276.70           290.30           282.33   
          -88.878548           277.30           290.27           282.37   
          -88.795219           277.47           289.13           281.13   
          -88.711891           276.70           289.13           282.43   
...                               ...              ...              ...   
66.623306 -61.629639            26.72            34.82            48.25   
          -61.546310            19.67            29.57            47.32   
          -61.462978            21.84            33.30            49.47   
          -61.379650            23.17            34.82            50.26   
          -61.296322            36.26            45.48            58.48   

                      t2080_rcp85_p50  t2050_rcp85_p90  t2080_rcp85_p90  
lat       lon                                                            
40.041039 -89.045212           303.87           297.30           317.67  
          -88.961884           304.07           297.20           317.53  
          -88.878548           304.07           297.20           317.47  
          -88.795219           304.60           296.20           316.10  
          -88.711891           304.87           295.53           316.27  
...                               ...              ...              ...  
66.623306 -61.629639            62.40            69.71            84.24  
          -61.546310            61.43            66.39            82.93  
          -61.462978            61.80            69.29            83.79  
          -61.379650            62.55            69.75            83.00  
          -61.296322            70.78            76.17            90.43  

[76206 rows x 12 columns]

# Merge both hist_50 and get the mean

In [20]:
dftgh["hist_p50"] = round((dftgh["hist_rcp45_p50"]+ dftgh["hist_rcp85_p50"])/2, 2)
dftgh2 = dftgh.reset_index()

A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  dftgh["hist_p50"] = round((dftgh["hist_rcp45_p50"]+ dftgh["hist_rcp85_p50"])/2, 2)



In [21]:
dftgh3 = dftgh2[["lat", "lon", "hist_p50"]]
dftgh3

,lat,lon,hist_p50
0,40.041039,-89.045212,255.10
1,40.041039,-88.961884,254.44
2,40.041039,-88.878548,254.22
3,40.041039,-88.795219,253.85
4,40.041039,-88.711891,254.03
...,...,...,...
76201,66.623306,-61.629639,38.64
76202,66.623306,-61.546310,36.10
76203,66.623306,-61.462978,36.86
76204,66.623306,-61.379650,38.25


# Merge means with the 45 and 85 DF

In [22]:
dftg_all = pd.merge(dftgh3, dftg1, on=["lat","lon"])

In [23]:
dftg_all

,lat,lon,hist_p50,t2050_rcp45_p10,t2080_rcp45_p10,t2050_rcp45_p50,t2080_rcp45_p50,t2050_rcp45_p90,t2080_rcp45_p90,t2050_rcp85_p10,t2080_rcp85_p10,t2050_rcp85_p50,t2080_rcp85_p50,t2050_rcp85_p90,t2080_rcp85_p90
0,40.041039,-89.045212,255.10,266.38,269.31,271.37,279.70,287.71,290.23,276.70,290.30,282.30,303.87,297.30,317.67
1,40.041039,-88.961884,254.44,265.77,269.27,271.17,279.63,287.65,290.40,276.70,290.30,282.33,304.07,297.20,317.53
2,40.041039,-88.878548,254.22,265.70,269.24,270.90,279.63,287.71,290.39,277.30,290.27,282.37,304.07,297.20,317.47
3,40.041039,-88.795219,253.85,266.11,268.17,270.20,279.33,287.93,290.14,277.47,289.13,281.13,304.60,296.20,316.10
4,40.041039,-88.711891,254.03,265.31,268.28,271.23,278.77,286.49,290.78,276.70,289.13,282.43,304.87,295.53,316.27
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
76201,66.623306,-61.629639,38.64,32.51,24.05,48.56,53.31,67.11,64.54,26.72,34.82,48.25,62.40,69.71,84.24
76202,66.623306,-61.546310,36.10,26.65,19.73,42.81,47.12,65.65,59.60,19.67,29.57,47.32,61.43,66.39,82.93
76203,66.623306,-61.462978,36.86,30.98,23.85,47.62,51.06,67.05,62.40,21.84,33.30,49.47,61.80,69.29,83.79
76204,66.623306,-61.379650,38.25,31.34,24.34,47.96,51.25,68.02,62.53,23.17,34.82,50.26,62.55,69.75,83.00


In [24]:
dftg_all.to_feather("/home/mlopez/EXEC/Processed data to clip with regions/"+var+"_annual.feather")

In [25]:
#dftg_all = pd.read_feather("/home/mlopez/EXEC/Processed data to clip with regions/"+var+"_annual.feather")

In [26]:
dftg_all 

,lat,lon,hist_p50,t2050_rcp45_p10,t2080_rcp45_p10,t2050_rcp45_p50,t2080_rcp45_p50,t2050_rcp45_p90,t2080_rcp45_p90,t2050_rcp85_p10,t2080_rcp85_p10,t2050_rcp85_p50,t2080_rcp85_p50,t2050_rcp85_p90,t2080_rcp85_p90
0,40.041039,-89.045212,255.10,266.38,269.31,271.37,279.70,287.71,290.23,276.70,290.30,282.30,303.87,297.30,317.67
1,40.041039,-88.961884,254.44,265.77,269.27,271.17,279.63,287.65,290.40,276.70,290.30,282.33,304.07,297.20,317.53
2,40.041039,-88.878548,254.22,265.70,269.24,270.90,279.63,287.71,290.39,277.30,290.27,282.37,304.07,297.20,317.47
3,40.041039,-88.795219,253.85,266.11,268.17,270.20,279.33,287.93,290.14,277.47,289.13,281.13,304.60,296.20,316.10
4,40.041039,-88.711891,254.03,265.31,268.28,271.23,278.77,286.49,290.78,276.70,289.13,282.43,304.87,295.53,316.27
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
76201,66.623306,-61.629639,38.64,32.51,24.05,48.56,53.31,67.11,64.54,26.72,34.82,48.25,62.40,69.71,84.24
76202,66.623306,-61.546310,36.10,26.65,19.73,42.81,47.12,65.65,59.60,19.67,29.57,47.32,61.43,66.39,82.93
76203,66.623306,-61.462978,36.86,30.98,23.85,47.62,51.06,67.05,62.40,21.84,33.30,49.47,61.80,69.29,83.79
76204,66.623306,-61.379650,38.25,31.34,24.34,47.96,51.25,68.02,62.53,23.17,34.82,50.26,62.55,69.75,83.00


# Merge complete DF with Polygons for each spatial scale

## Dictionary - regioins: column name, short name

In [27]:
short_dict = {"DDE_STF_20K_REG_FOR_VUE_S": ("NM_REG_FOR", "RF"), 
              "DDE_STF_20K_UA_PER_VUE_S": ("PER_NO_UA", "UA"), 
              "DOM_BIO": ("NOM", "DB"), 
              "REG_ECO": ("NOM", "RE"), 
              "SDOM_BIO": ("NOM", "SDB"), 
              "Secteurs_Operations_Regionales": ("D_GENERAL", "SOR"), 
              "SREG_ECO": ("NOM", "SRE"), 
              "territoire_guide": ("TER_GUIDE", "TG") 
              }

for region, (name, short) in short_dict.items():
    print(region, name, short)

DDE_STF_20K_REG_FOR_VUE_S NM_REG_FOR RF
DDE_STF_20K_UA_PER_VUE_S PER_NO_UA UA
DOM_BIO NOM DB
REG_ECO NOM RE
SDOM_BIO NOM SDB
Secteurs_Operations_Regionales D_GENERAL SOR
SREG_ECO NOM SRE
territoire_guide TER_GUIDE TG


In [28]:
def replace_all(text, dic):
    for i, j in dic.items():
        text = text.replace(i, j)
    return text

In [29]:
d = { 'BAS-SAINT-LAURENT':"Bas-Saint-Laurent", 
     'SAGUENAY -LAC-SAINT-JEAN': "Saguenay -Lac-Saint-Jean",
     'CAPITALE-NATIONALE-CHAUDIÈRE-APPALACHES':"Capitale-Nationale-Chaudiere-Appalaches",
     'MAURICIE-CENTRE-DU-QUÉBEC':'Mauricie-Centre-du-Quebec','OUTAOUAIS':'Outaouais', 
     'ABITIBI-TEMISCAMINGUE':'Abitibi-Temiscamingue', 'COTE-NORD':'Cote-Nord',
     'NORD-DU-QUEBEC':'Nord-Du-Quebec', 'GASPESIE-ILES-DE-LA-MADELEINE':'Gaspesie-Iles-De-La-Madeleine', 
     'LANAUDIERE':'Lanaudiere','LAURENTIDES':"Laurentides", 
     "ESTRIE-MONTÉRÉGIE-LAVAL-MONTRÉAL":"Estrie-Monteregie-Laval-Montreal", 
     "é": "e", "à": "a", "è": "e", "ô" : "o", "ç":"c", "É": "E", "È": "E", "Î": "i", "Ç":"C" } 

In [30]:
for region, (name, short) in short_dict.items():
    print(region)
    dfpolyshape = from_geofeather('/home/mlopez/EXEC/Grids-polygons-regions/Grid-'+region+'.feather')
    dftp = pd.merge(dftg_all, dfpolyshape, on=["lat","lon"])
    print ("Merged with polygons")
    listTG = []
    for tg in dftp[name].unique().tolist():
        df2 = dftp[dftp[name] == tg]
        print (tg)
        if tg != None:
            listTG.append(df2)
        #print (listTG)
    for df in listTG:
        geometry = df["geometry"]
        crs = {'init': "epsg:4326"}
        gdf = GeoDataFrame(df, crs=crs, geometry=geometry)
        print (gdf[name].iloc[0])
        #Substitute filename accents
        gdf.to_file(replace_all("/home/mlopez/EXEC/GeoJsonMFFP/"+gdf[name].iloc[0], d)+"_"+var+"_annual.json", driver="GeoJSON")
    geometry = dftp["geometry"]
    crs = {'init': "epsg:4326"}
    gdf = GeoDataFrame(dftp, crs=crs, geometry=geometry)
    gdf.to_file("/home/mlopez/EXEC/GeoJsonMFFP/"+short+"_"+var+"_annual.json", driver="GeoJSON")

  warnings.warn(



DDE_STF_20K_REG_FOR_VUE_S
Merged with polygons
ESTRIE-MONTÉRÉGIE-LAVAL-MONTRÉAL
OUTAOUAIS
LAURENTIDES
MAURICIE-CENTRE-DU-QUÉBEC
LANAUDIERE
CAPITALE-NATIONALE-CHAUDIÈRE-APPALACHES
ABITIBI-TEMISCAMINGUE
BAS-SAINT-LAURENT
GASPESIE-ILES-DE-LA-MADELEINE
SAGUENAY -LAC-SAINT-JEAN
COTE-NORD
NORD-DU-QUEBEC
ESTRIE-MONTÉRÉGIE-LAVAL-MONTRÉAL
OUTAOUAIS
LAURENTIDES
MAURICIE-CENTRE-DU-QUÉBEC
LANAUDIERE
CAPITALE-NATIONALE-CHAUDIÈRE-APPALACHES
ABITIBI-TEMISCAMINGUE
BAS-SAINT-LAURENT
GASPESIE-ILES-DE-LA-MADELEINE
SAGUENAY -LAC-SAINT-JEAN
COTE-NORD
NORD-DU-QUEBEC


  warnings.warn(



DDE_STF_20K_UA_PER_VUE_S
Merged with polygons
05151
07151
07251
06151
03571
06452
06271
07351
07152
08151
06471
04151
03451
08152
07352
03171
07451
04352
04251
01171
03153
08351
02371
03351
08251
04351
01272
08451
02571
11161
09751
11262
08462
08651
02651
08751
08551
02471
11263
08762
02751
02664
09351
08652
02666
08763
09471
08764
02665
08665
02663
08562
08664
08666
02662
08663
09551
02661
09352
05151
07151
07251
06151
03571
06452
06271
07351
07152
08151
06471
04151
03451
08152
07352
03171
07451
04352
04251
01171
03153
08351
02371
03351
08251
04351
01272
08451
02571
11161
09751
11262
08462
08651
02651
08751
08551
02471
11263
08762
02751
02664
09351
08652
02666
08763
09471
08764
02665
08665
02663
08562
08664
08666
02662
08663
09551
02661
09352


  warnings.warn(



...

Massif des monts Groulx


ERROR:fiona._env:/home/mlopez/EXEC/GeoJsonMFFP/Coteaux des lacs Matonipi et Jonquet_growing_season_length_annual.json: No such file or directory


Coteaux des lacs Matonipi et Jonquet


  warnings.warn(



territoire_guide
Merged with polygons
1a
2c
3d
2b
2a
3c
3ab
4bc
4de
5ef
4f
4a
5jk
5bcd
5a
4gh
5hi
5g
6cdefg
6hi
6ab
6j
6mn
6kl
6opqr
1a
2c
3d
2b
2a
3c
3ab
4bc
4de
5ef
4f
4a
5jk
5bcd
5a
4gh
5hi
5g
6cdefg
6hi
6ab
6j
6mn
6kl
6opqr
